In [ ]:
from qiskit_nature.units import DistanceUnit
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import SLSQP
from qiskit.primitives import Estimator
from qiskit_nature.second_q.circuit.library import HartreeFock, UCCSD
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit.transpiler import generate_preset_pass_manager
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2


service = QiskitRuntimeService()
backend = service.least_busy(simulator=False, operational=True, min_num_qubits=4)

# Molecular system
driver = PySCFDriver(
    atom="H 0 0 0; H 0 0 0.735",
    basis="sto3g",
    charge=0,
    spin=0,
    unit=DistanceUnit.ANGSTROM,
)
es_problem = driver.run()

# Fermionic Hamiltonian
ferm_hamiltonian = es_problem.hamiltonian.second_q_op()

# Map to qubits
mapper = JordanWignerMapper()
qubit_op = mapper.map(ferm_hamiltonian)
print(qubit_op)

# Ansatz: UCCSD with HF initial state
ansatz = UCCSD(
    es_problem.num_spatial_orbitals,
    es_problem.num_particles,
    mapper,
    initial_state=HartreeFock(
        es_problem.num_spatial_orbitals,
        es_problem.num_particles,
        mapper,
    ),
)


# VQE setup
vqe = VQE(estimator=Estimator(), ansatz=ansatz, optimizer=SLSQP())
vqe.initial_point = [0.0] * ansatz.num_parameters

# Run VQE
result = vqe.compute_minimum_eigenvalue(qubit_op)

# Extract results
E_elec = result.eigenvalue.real
E_total = E_elec + es_problem.nuclear_repulsion_energy

print(f"Electronic energy: {E_elec:.6f} Hartee")
print(f"Nuclear repulsion: {es_problem.nuclear_repulsion_energy:.6f} Hartee")
print(f"Total energy:     {E_total:.6f} Hartee")

# Draw ansatz
ansatz.draw("mpl")

ImportError: cannot import name 'BaseSampler' from 'qiskit.primitives' (/Users/ema/Documents/Python/quantum/Quantum-Examples/.venv/lib/python3.11/site-packages/qiskit/primitives/__init__.py)